In [12]:
from clean import clean_mi_pl_bs, clean_mi_deferrals, clean_gross_tp_walk, clean_unearned_premium_unexpired_risk, clean_s2_summary_sheet, clean_version_control_solvency_balance_sheet, clean_version_control_scr_review, clean_version_control_s2_gaap, clean_version_control_s2_cap_prov_summary
import pandas as pd
import math
import numpy as np

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', None)

In [32]:
all_s2_tables = clean_s2_summary_sheet(file_path = 'documents\\2506 MICL_S2 Gross TP walk.xlsx', sheet_name='S2 Balance Sheet Summary')
scr_summary = all_s2_tables['scr']
mi_bs = clean_mi_pl_bs(file_path = 'documents\\Jun25 MICL MI Pack v3.1 Isi pack exc PC.xlsx', sheet_name='BS')
solvency_balance_sheet_liabilities = clean_version_control_solvency_balance_sheet(file_path = 'documents\\2506 MICL Version Control.xlsx', skip_rows = 55, n_rows = 34, element = 'Liabilities')
# op_risk_calc = all_s2_tables['op_risk']

<string>:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Currency Risk

In [35]:
usd_bs = ['Dayim Loan (USD Purchase)', 'WSL (8VC) Shares (USD purchase)', 'CCA Longevity III DAC (USD)', 'Perceptive', 'RBS USD Account', 'JSS Forwards']
usd_list = []
for usd in usd_bs:
    usd_list.append(mi_bs.loc[mi_bs['Description']== usd, '2025-06-01 00:00:00 Actual'].item())

usd_list[-1] *= -1

currency_net_assets = sum(usd_list)

usd_forward = solvency_balance_sheet_liabilities.loc[solvency_balance_sheet_liabilities['']== 'Derivatives', 'Col N BS SII BS 2025-06-01 00:00:00'].item()

currency_risk_exposure = currency_net_assets - usd_forward

currency_scr = 0.25 * currency_risk_exposure

currency_scr_provided = scr_summary.loc[scr_summary['Item'] == 'Currency', 'Final - As at 30th June 2025'].item()
print(usd_forward)
print(currency_net_assets)
print(currency_scr, currency_scr_provided)

13895084.22
12558987.14
-334024.27 40373.94749999978


In [28]:
currency_risk_exposure

12558987.14

Interest rate risk: those assets and liabilities are exposed to movements in the term structure of interest rates:
- assets stressed within the spread risk sub-module (i.e. bonds and loans)
- technical provisions

Trying to get yield curve from BoE

In [9]:
# import pandas as pd
# import requests
# import io
# import matplotlib.pyplot as plt
# import matplotlib.ticker as mticker

# # --- 1. Define Request Parameters ---

# # ✨ UPDATED: The old IUDZA... codes are obsolete. These are the new, correct codes.
# SERIES_CODES = [
#     'CFCMG01', 'CFCMG02', 'CFCMG03', 'CFCMG04', 'CFCMG05', 'CFCMG06', 'CFCMG07',
#     'CFCMG08', 'CFCMG09', 'CFCMG10', 'CFCMG11', 'CFCMG12', 'CFCMG13', 'CFCMG14',
#     'CFCMG15', 'CFCMG16', 'CFCMG17', 'CFCMG18', 'CFCMG19', 'CFCMG20', 'CFCMG21',
#     'CFCMG22', 'CFCMG23', 'CFCMG24'
# ]
# start_date = "01/Jun/2025"
# end_date = "30/Jun/2025"

# # --- 2. Construct URL and Fetch Data ---
# base_url = "https://www.bankofengland.co.uk/boeapps/database/fromshowcolumns.asp"
# params = {
#     "Datefrom": start_date, "Dateto": end_date, "SeriesCodes": ",".join(SERIES_CODES),
#     "CSVF": "TN", "UsingCodes": "Y", "VPD": "Y", "VFD": "N", "xml.x": "yes"
# }
# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
# }

# print("Fetching data from the Bank of England...")

# try:
#     response = requests.get(base_url, params=params, headers=headers)
#     response.raise_for_status()
    
#     # --- 3. Robust Parsing Logic ---
#     raw_csv_text = response.text
#     lines = raw_csv_text.strip().split('\n')
#     header_row_index = -1
#     for i, line in enumerate(lines):
#         # The new header is "DATE" in uppercase
#         if line.strip().upper().startswith('DATE'):
#             header_row_index = i
#             break
            
#     if header_row_index == -1:
#         # Add a debug print to see what the server is actually sending
#         print("\n--- SERVER RESPONSE (first 500 chars) ---")
#         print(raw_csv_text[:500])
#         print("-------------------------------------------\n")
#         raise ValueError("Could not find the header row starting with 'DATE' in the CSV data.")

#     csv_data = io.StringIO(raw_csv_text)
#     yield_curve_df = pd.read_csv(csv_data, header=header_row_index)

#     # --- 4. Clean and Display Data ---
#     # The date column is now named 'DATE'
#     yield_curve_df.rename(columns={'DATE': 'Date'}, inplace=True)
#     yield_curve_df['Date'] = pd.to_datetime(yield_curve_df['Date'], format='%d %b %Y')
#     yield_curve_df.set_index('Date', inplace=True)

#     # The column names are now the series codes, we need to map them to maturities
#     maturities_map = {
#         'CFCMG01': '0.5 YR', 'CFCMG02': '1 YR', 'CFCMG03': '2 YR', 'CFCMG04': '3 YR',
#         'CFCMG05': '4 YR', 'CFCMG06': '5 YR', 'CFCMG07': '6 YR', 'CFCMG08': '7 YR',
#         'CFCMG09': '8 YR', 'CFCMG10': '9 YR', 'CFCMG11': '10 YR', 'CFCMG12': '11 YR',
#         'CFCMG13': '12 YR', 'CFCMG14': '13 YR', 'CFCMG15': '14 YR', 'CFCMG16': '15 YR',
#         'CFCMG17': '16 YR', 'CFCMG18': '17 YR', 'CFCMG19': '18 YR', 'CFCMG20': '19 YR',
#         'CFCMG21': '20 YR', 'CFCMG22': '25 YR', 'CFCMG23': '30 YR', 'CFCMG24': '40 YR'
#     }
#     yield_curve_df.rename(columns=maturities_map, inplace=True)

#     for col in yield_curve_df.columns:
#         yield_curve_df[col] = pd.to_numeric(yield_curve_df[col], errors='coerce')

#     print("\n✅ Success! GBP yield curve for June 2025:")
#     print(yield_curve_df.head())

# except requests.exceptions.RequestException as e:
#     print(f"\n❌ Error fetching data: {e}")
# except Exception as e:
#     print(f"\n❌ An error occurred during data processing: {e}")

Fetching data from the Bank of England...

--- SERVER RESPONSE (first 500 chars) ---
<!DOCTYPE html>
<html lang="en" class="no-js">
    <head>
    <meta charset="utf-8">
    <meta http-equiv="Content-type" content="text/html; charset=utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="format-detection" content="telephone=no">
    <title>Error</title>
    <link rel="shortcut icon" href="favicon.ico" type="image/x-icon">
<link rel="stylesheet" href="assets/css/layout.
-------------------------------------------


❌ An error occurred during data processing: Could not find the header row starting with 'DATE' in the CSV data.


Construct Operational

- **Inputs:**
    - **Gross Earned Premium (current year & prior year):** Sourced from `PL`.
    - **Gross Technical Provisions (non-life):** Sourced from `BS`.
    - **The BSCR:** The diversified sum of the Market, Counterparty, and Underwriting risk modules, calculated in the preceding step of the model.
    
- Calculation:
    1. **Calculate the Premium-Based Component (`Op_Prem`):** This step calculates a value based on the insurer's earned premiums and its recent growth.
        - **Formula:** `Op_Prem = 3% × GEP_CY + Max[0, 0.3% × (GEP_CY - 1.2 × GEP_PY)]`.
        - This means the model takes **3% of the gross earned premium from the last 12 months** and adds a small additional amount if premium volume has grown by more than 20% compared to the prior year, capturing the operational risk associated with rapid growth.
        
    2. **Calculate the Provisions-Based Component (`Op_Prov`):** This step calculates a value based on the size of the insurer's technical provisions.
        - **Formula:** `Op_Prov = 3% × Max(0, TP_Non-Life)`.
        - For a non-life insurer, this is **3% of the gross non-life technical provisions**.
        
    3. **Determine the Final SCR:** The model then combines these values with the BSCR.
        - **Formula:** `SCR_Operational = Min(0.3 × BSCR, Max(Op_Prem, Op_Prov))`.
        - The logic is to first take the **larger** of the premium-based and provisions-based components. This result is then compared to **30% of the BSCR**. The final Operational Risk SCR is the **smaller** of these two figures.

In [34]:
# op_risk = op_risk_calc.loc[op_risk_calc['OP risk calc'] == 'OP risk', 'Final - As at 30th June 2025'].item()

# gep_cy = op_risk_calc.loc[op_risk_calc['OP risk calc'] == 'GEP - historic 1 to 12 months', 'Final - As at 30th June 2025'].item() # gross earned premium in the past 12 months
# gep_py = op_risk_calc.loc[op_risk_calc['OP risk calc'] == 'GEP - historic 13 - 24 months', 'Final - As at 30th June 2025'].item() # gross earned premiums in the 12 months preceding the past 12 months
# op_prem = 0.03 * gep_cy + max(0.03 * (gep_cy - 1.2 * gep_py), 0)

# tp_non_life = op_risk_calc.loc[op_risk_calc['OP risk calc'] == 'Gross TPs', 'Final - As at 30th June 2025'].item() # technical provisions recorded in the non-life underwriting module
# tp_life = 0 # technical provisions recorded in the life underwriting module
# op_prov = 0.03 * max(0, tp_non_life) + 0.045 * max(0, tp_life)

# calc_operational = min(0.3 * final_bscr, max(op_prem, op_prov))

# if math.isclose(calc_operational, op_risk):
#     print('Operational Risk Checks Out')
# else:
#     print('Operational Risk Does Not Check Out')

Source `gep_cy`, `gep_py` and `tp_non_life` myself.

In [35]:
# mi_pl = clean_mi_pl_bs(file_path = 'documents\\Jun25 MICL MI Pack v3.1 Isi pack exc PC.xlsx', sheet_name='PL')


Gross Written Premium 1 to 12 months

In [36]:
# gross_written_premium = mi_pl.loc[mi_pl[('', 'Description')] == 'Gross written premium', ('YTD', 'Actual')]
# gross_written_premium = gross_written_premium[:-1].item()
# change_gross_provision_unearned_premiums = mi_pl.loc[mi_pl[('', 'Description')] == 'Change in the gross provision for unearned premiums', ('YTD', 'Actual')].item()
# gross_earned_premium = gross_written_premium - change_gross_provision_unearned_premiums
# print(gross_earned_premium)

Gross Written Premium 13 to 24 months

In [37]:
# gross_written_premium_past = mi_pl.loc[mi_pl[('', 'Description')] == 'Gross written premium', ('', 'Prior_Year')]
# gross_written_premium_past = gross_written_premium_past[:-1].item()
# change_gross_provision_unearned_premiums_past = mi_pl.loc[mi_pl[('', 'Description')] == 'Change in the gross provision for unearned premiums', ('', 'Prior_Year')].item()
# gross_earned_premium_past = gross_written_premium_past - change_gross_provision_unearned_premiums_past
# print(gross_earned_premium_past)

Gross Technical Provisions

In [38]:
# mi_bs = clean_mi_pl_bs(file_path = 'documents\\Jun25 MICL MI Pack v3.1 Isi pack exc PC.xlsx', sheet_name='BS')

In [39]:
# provision_for_unearned_premiums = mi_bs.loc[mi_bs[('', 'Description')] == 'Provision for unearned premiums, gross amount', ('2025-06-01 00:00:00', 'Actual')].item()
# claims_outstanding = mi_bs.loc[mi_bs[('', 'Description')] == 'Claims outstanding, gross amount', ('2025-06-01 00:00:00', 'Actual')].item()
# ibnr = mi_bs.loc[mi_bs[('', 'Description')] == 'IBNR - Gross Amount', ('2025-06-01 00:00:00', 'Actual')].item()
# gross_technical_provisions = provision_for_unearned_premiums + claims_outstanding + ibnr
# print(gross_technical_provisions)

Check to ensure calcualtion is correct

In [40]:
def run_check(name: str, calculated_value: float, provided_value: float):
    """Compares a calculated value to a provided value and prints the result."""
    if provided_value is None:
        print(f"⚠️ {name}: Cannot check, provided value not found.")
        return

    if math.isclose(calculated_value, provided_value):
        print(f"✅ {name}: Checks out.")
    else:
        diff = abs(calculated_value - provided_value)
        # Using an f-string to format the number with commas and 2 decimal places
        print(f"❌ {name}: Does NOT check out. Difference is {diff:,.2f}.")

Function SCR Module

In [41]:
def scr_module(correlation_matrix, risk_type, labels):

    value_column = 'Final - As at 30th June 2025'

    data = scr_summary[
        (scr_summary['SCR'] == risk_type) &
        (scr_summary['Item'].isin(labels))
    ].set_index('Item') # Set 'Item' as the index to allow ordering

    # Reindex the data to match the labels order, ensuring the vector is correct; extract the column of values and convert it to a NumPy array.
    vector = data.reindex(labels)[value_column].to_numpy()

    # vector.T @ correlation_matrix: weigh each SCR value by its correlation to all other risks & dot product with vector
    variance = vector.T @ correlation_matrix @ vector
    calc_scr = np.sqrt(variance) # bring back to original currency unit

    calc_diversification = calc_scr - sum(vector)

    validation_items = ['Total', 'Diversification']

    provided_values = scr_summary[
        (scr_summary['SCR'] == risk_type) &
        (scr_summary['Item'].isin(validation_items))
    ].set_index('Item')[value_column] # Creates a Pandas Series indexed by 'Item'

    scr_provided = provided_values.get('Total')
    diversification_scr_provided = provided_values.get('Diversification')

    print(f"--- {risk_type} Risk Validation ---")
    run_check(name=f"{risk_type} SCR",
            calculated_value=calc_scr,
            provided_value=scr_provided)

    run_check(name="Diversification Benefit",
            calculated_value=calc_diversification,
            provided_value=diversification_scr_provided)

    return calc_scr

Call SCR function

In [42]:
A = 0.5
# Source: COMMISSION DELEGATED REGULATION (EU) 2015/35 of 10 October 2014, Article 164, Page 105 of Journal of the European Union (L 12/1)
market_scr = scr_module(
    correlation_matrix = [
    [1,    A,    A,    A,    0, 0.25],    # Interest rate
    [A,    1, 0.75, 0.75,    0, 0.25],    # Equity
    [A, 0.75,    1,  0.5,    0, 0.25],    # Property
    [A, 0.75,  0.5,    1,    0, 0.25],    # Spread
    [0,    0,    0,    0,    1,    0],     # Concentration
    [0.25, 0.25, 0.25, 0.25, 0,    1]     # Currency
], risk_type = 'Market', labels = ['Interest rate', 'Equity', 'Property', 'Spread', 'Concentration', 'Currency'])

counterparty_scr = scr_module(
    correlation_matrix = [
    [1, 0.75],  # Type 1
    [0.75, 1]  # Type 2
], risk_type = 'Counterparty', labels = ['Type 1', 'Type 2'])

nonlife_scr = scr_module(
    correlation_matrix = [
    [1.00, 0.25, 0.00],  # Premium/Reserve
    [0.25, 1.00, 0.00],  # Catastrophe
    [0.00, 0.00, 1.00]   # Lapse
], risk_type = 'Non-life', labels = ['Premium  & reserve risk', 'Cat risk', 'Lapse risk'])

--- Market Risk Validation ---
✅ Market SCR: Checks out.
✅ Diversification Benefit: Checks out.
--- Counterparty Risk Validation ---
✅ Counterparty SCR: Checks out.
✅ Diversification Benefit: Checks out.
--- Non-life Risk Validation ---
✅ Non-life SCR: Checks out.
✅ Diversification Benefit: Checks out.


BSCR

In [52]:
labels = ['market_scr', 'counterparty_scr', 'nonlife_scr'] # set order for risks; the correlation matrix is built with this order in mind

correlation_matrix_bscr = np.array([
    [1.00, 0.25, 0.25],
    [0.25, 1.00, 0.50],
    [0.25, 0.50, 1.00]
])

scr_values = {
    'market_scr': market_scr,
    'counterparty_scr': counterparty_scr,
    'nonlife_scr': nonlife_scr
}

bscr_vector = np.array([scr_values[label] for label in labels]) # build bscr_vector in order required to match correlation matrix

# BSCR = SQRT(Σi,j Corr(i,j) * SCRi * SCRj) can be expressed in matrix notation as: BSCR = SQRT(bscr_vector^T * Correlation_Matrix * bscr_vector) where ^T denotes the transpose of the vector.
variance = bscr_vector.T @ correlation_matrix_bscr @ bscr_vector
calc_bscr = np.sqrt(variance) # brings back to original currency unit
calc_diversification = calc_bscr - sum(bscr_vector)

bscr_provided = scr_summary.loc[scr_summary['SCR'] == 'Basic SCR', 'Final - As at 30th June 2025'].item()
diversification_provided = scr_summary.loc[scr_summary['SCR'] == 'Diversification', 'Final - As at 30th June 2025'].item()

run_check(name = 'BSCR', calculated_value = calc_bscr, provided_value = bscr_provided)
run_check(name = 'BSCR Diversification', calculated_value = calc_diversification, provided_value = diversification_provided)

✅ BSCR: Checks out.
✅ BSCR Diversification: Checks out.
